In [ ]:
import pandas as pd
path1 = r'uid_relate.xlsx'
data1 = pd.read_excel(path1, engine='openpyxl')
path2 = r'translations.xlsx'
data2 = pd.read_excel(path2, engine='openpyxl')
path4 = r'country.xlsx'
data4 = pd.read_excel(path4, engine='openpyxl')

In [ ]:
def string_to_list_v2(input_string):
    input_string = input_string.strip("[]")
    return [int(num_str) for num_str in input_string.split(",")]

def filter_1st_element(df):
    df_list = df.tolist()
    v = None
    for item in df_list:
        if item == '-':
            continue
        else:
            v = item
            break
    return v

def filter_df_tostring(df):
    strrr = []
    for i in df.tolist():
        strrr.append(i)
    return strrr

columns1 = data1.columns

point_uid = []
routids = []
directids = []
routtype = []
routid = 0

for index, row in data1.iterrows():
    for column in columns1:
        if column != "unique_id" and row[column] != "['']":
            str_2_uid = string_to_list_v2(row[column])
            for i in str_2_uid:
                point_uid.append(row['unique_id'])
                routids.append(routid)
                directids.append(0)
                point_uid.append(i)
                routids.append(routid)
                directids.append(1)
                routtype.append(column)
                routtype.append(column)
                routid = routid + 1
                
                
point_lon = []
point_lat = []
point_time = []
point_language = []
point_country = []
point_type = []
point_iso = []


for j in range(len(point_uid)):
    uid1 = point_uid[j]
    condition1 = data2['unique_id'] == uid1
    filtered_lan_1 = filter_1st_element(data2[condition1]['language'])
    filtered_pubtime_1 = filter_1st_element(data2[condition1]['publishyear_1st'])
    filtered_publocation_1 = filter_1st_element(data2[condition1]['country'])
    filtered_type_1 = filter_1st_element(data2[condition1]['type'])
    filtered_lat_1 = filter_1st_element(data2[condition1]['lat'])
    filtered_lon_1 = filter_1st_element(data2[condition1]['lon'])
    filtered_iso_1 = filter_1st_element(data2[condition1]['country_code_iso2'])
  
    point_lon.append(filtered_lon_1)
    point_lat.append(filtered_lat_1)
    point_time.append(filtered_pubtime_1)
    point_language.append(filtered_lan_1)
    point_country.append(filtered_publocation_1)
    point_type.append(filtered_type_1)
    point_iso.append(filtered_iso_1)

data3 = pd.DataFrame()
data3['point_uid'] = point_uid
data3['routids'] = routids
data3['point_direct'] = directids
data3['point_lon'] = point_lon
data3['point_lat'] = point_lat
data3['point_time'] = point_time
data3['point_language'] = point_language
data3['point_country'] = point_country
data3['point_type'] = point_type
data3['point_iso'] = point_iso
data3['rout_type'] = routtype

point_iso_3 = []
point_cn = []
for iso in point_iso:
    mark = 0
    for index, row in data4.iterrows():
        if row['iso2'] == iso:
            point_iso_3.append(row['iso3'])
            point_cn.append(row['cn'])
            mark = 1
            break
    if mark == 0:
        point_iso_3.append("-")
        point_cn.append("-")
        
data3['point_iso_3'] = point_iso_3
data3['point_cn'] = point_cn

In [ ]:
start_points = data3[data3['point_direct'] == 0][['routids', 'point_country', 'point_iso', 'point_iso_3', 'point_cn']].rename(columns={'point_country': 'Start Country'})
end_points = data3[data3['point_direct'] == 1][['routids', 'point_country', 'point_iso', 'point_iso_3', 'point_cn']].rename(columns={'point_country': 'End Country'})
rout_type = data3[data3['point_direct'] == 1][['routids', 'rout_type']]

routes = pd.merge(start_points, end_points, on='routids')
routes = pd.merge(routes, rout_type, on='routids')

route_data = routes.groupby(['Start Country', 'End Country','point_iso_x', 'point_iso_y','point_iso_3_x', 'point_iso_3_y','point_cn_x','point_cn_y','rout_type']).size().reset_index(name='Count')
route_data = route_data.pivot_table(index=['Start Country', 'End Country','point_iso_x', 'point_iso_y','point_iso_3_x', 'point_iso_3_y','point_cn_x','point_cn_y'], columns="rout_type", values="Count", aggfunc="sum", fill_value=0).reset_index()
route_data["Intensity"] = 1 * route_data["retranslation"] + 0.7 * route_data["source text"]+ 0.3 * route_data["reference"] 
route_data.columns.name = None

In [ ]:
all_countries = pd.concat([route_data['point_iso_3_x'], route_data['point_iso_3_y']]).unique()
adjacency_matrix_full = pd.DataFrame(index=all_countries, columns=all_countries).fillna(0)
for index, row in route_data.iterrows():
    if row['point_iso_3_x'] != row['point_iso_3_y']:
        adjacency_matrix_full.loc[row['point_iso_3_x'], row['point_iso_3_x']] += row["Intensity"]

rows_to_drop = adjacency_matrix_full.index[(adjacency_matrix_full == 0).all(axis=1)]
cols_to_drop = adjacency_matrix_full.columns[(adjacency_matrix_full == 0).all(axis=0)]

indices_to_drop = rows_to_drop.intersection(cols_to_drop)
adjacency_matrix_full = adjacency_matrix_full.drop(index=indices_to_drop, columns=indices_to_drop)

csv_file_path = r'adjacency_matrix.csv'
adjacency_matrix_full.to_csv(csv_file_path, encoding='utf-8-sig')